# Aviation Accident Risk Analysis for Strategic Investment

## Goals

To help the company identify the lowest-risk aircraft models for commercial and private aviation.  
The goal is to analyze historical accident data to uncover patterns that can guide safe aircraft investments.


## Data

The dataset is from the National Transportation Safety Board (NTSB), covering civil aviation accidents from 1962 to 2023.  
The data includes accident details such as aircraft model, date, location, phase of flight, injury counts, and weather conditions.


## Methods

We used the CRISP-DM methodology:
- Cleaned and standardized key columns (e.g., date, injuries, aircraft make/model)
- Conducted univariate, bivariate, and multivariate analysis
- Aggregated injuries by aircraft model and purpose of flight
- Created visualizations to highlight risk patterns


## Results

- Most accidents occurred between 1990 and 2010.
- A small number of aircraft makes are involved in a majority of severe accidents.
- Training and private-use flights tend to have higher injury rates than commercial flights.
- Certain aircraft models consistently show low injury rates per incident.


### Business Recommendation 1

Invest in aircraft models with historically low total injuries per incident.  
Examples: [insert model names based on your EDA].


### Business Recommendation 2

Avoid older models and aircraft used predominantly in private or training settings.  
These showed a higher number of accidents and severe injuries.


### Business Recommendation 3

Start with commercial-use aircraft types with strong safety records, then expand based on risk-adjusted performance.  
Consider insurance and maintenance costs as part of the decision process.
